In [1]:
import numpy as np


class Task:
    def __init__(self, id, execution_time):
        self.id = id
        self.execution_time = execution_time


# Fitness function to calculate total turnaround time
def calculate_turnaround_time(schedule):
    turnaround_time = 0
    total_time = 0
    for task in schedule:
        total_time += task.execution_time
        turnaround_time += total_time  # Cumulative time
    return turnaround_time


# Initialize population
def initialize_population(tasks, population_size):
    return [np.random.permutation(tasks) for _ in range(population_size)]


# Selection: Tournament selection
def selection(population):
    tournament_size = 3
    selected_indices = np.random.choice(len(population), tournament_size, replace=False)
    selected = [population[i] for i in selected_indices]
    selected.sort(key=lambda x: calculate_turnaround_time(x))
    return selected[0]  # Best individual


# Crossover: Order crossover
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(np.random.choice(range(size), 2, replace=False))
    child = [None] * size
    child[start:end] = parent1[start:end]

    fill_pos = [i for i in range(size) if child[i] is None]
    fill_values = [item for item in parent2 if item not in child]

    for pos in fill_pos:
        child[pos] = fill_values.pop(0)

    return child


# Mutation: Swap mutation
def mutate(schedule, mutation_rate=0.1):
    if np.random.rand() < mutation_rate:
        i, j = np.random.choice(len(schedule), 2, replace=False)
        schedule[i], schedule[j] = schedule[j], schedule[i]
    return schedule


# Genetic Algorithm for FIFO Scheduling
def genetic_algorithm(tasks, population_size, generations):
    population = initialize_population(tasks, population_size)
    for _ in range(generations):
        new_population = []
        for _ in range(population_size):
            parent1 = selection(population)
            parent2 = selection(population)
            child = crossover(parent1, parent2)
            child = mutate(child)
            new_population.append(child)
        population = new_population
    # Return the best solution found
    best_schedule = min(population, key=lambda x: calculate_turnaround_time(x))
    return best_schedule, calculate_turnaround_time(best_schedule)


# Example usage
tasks = [Task(i, np.random.randint(1, 10)) for i in range(5)]  # Create 5 tasks with random execution times
best_schedule, best_time = genetic_algorithm(tasks, population_size=20, generations=100)


print("Best Schedule (Task IDs):", [task.id for task in best_schedule])
print("Total Turnaround Time:", best_time)

Best Schedule (Task IDs): [2, 1, 0, 4, 3]
Total Turnaround Time: 43
